In [1]:
!pip install pulp
from pulp import *

In [47]:
# Creates a list of the products
months = ['000','JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']

# A dictionary of the unit cost per month
cost = {'000': 0,
    'JAN': 240, 
         'FEB': 250,
       'MAR': 265, 
         'APR': 285,
       'MAY': 280, 
         'JUN': 260}

# A dictionary of the demand per month
demand = {'000': 0,
          'JAN': 1000, 
         'FEB': 4500,
       'MAR': 6000, 
         'APR': 5500,
       'MAY': 3500, 
         'JUN': 4000}

# A dictionary of the min production per month
minOut = {'000': 0,
          'JAN': 2000, 
         'FEB': 1750,
       'MAR': 2000, 
         'APR': 2250,
       'MAY': 2000, 
         'JUN': 1750}

# A dictionary of the max production per month
maxOut = {'JAN': 4000, 
         'FEB': 3500,
       'MAR': 4000, 
         'APR': 4500,
       'MAY': 4000, 
         'JUN': 3500,
         '000': 0}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Upton Corporation Production Planning", LpMinimize)

# A dictionary of the output variables
o = LpVariable.dicts("output",months,0)

# A dictionary of the storage variables
s = LpVariable.dicts("storage",months,1500,6000)

# Create objective function
prob += lpSum([o[i]*cost[i] for i in months]) + lpSum([0.015*cost[months[i+1]]*0.5*(s[months[i+1]]+s[months[i]]) for i in range(6)]), "Total Cost"

#Enforce values of init period
prob += s["000"] == 2750

# Create constraints
for month in months:
    prob += o[month] >= minOut[month], "minOut{}".format(month)
    prob += o[month] <= maxOut[month], "maxOut{}".format(month)

for i in range(6):
    prob += s[months[i]]+o[months[i+1]]-demand[months[i+1]]==s[months[i+1]]

In [48]:
print(prob)

Upton Corporation Production Planning:
MINIMIZE
285*output_APR + 250*output_FEB + 240*output_JAN + 260*output_JUN + 265*output_MAR + 280*output_MAY + 1.7999999999999998*storage_000 + 4.2375*storage_APR + 3.8625*storage_FEB + 3.675*storage_JAN + 1.95*storage_JUN + 4.125*storage_MAR + 4.05*storage_MAY + 0.0
SUBJECT TO
_C1: storage_000 = 2750

minOut000: output_000 >= 0

maxOut000: output_000 <= 0

minOutJAN: output_JAN >= 2000

maxOutJAN: output_JAN <= 4000

minOutFEB: output_FEB >= 1750

maxOutFEB: output_FEB <= 3500

minOutMAR: output_MAR >= 2000

maxOutMAR: output_MAR <= 4000

minOutAPR: output_APR >= 2250

maxOutAPR: output_APR <= 4500

minOutMAY: output_MAY >= 2000

maxOutMAY: output_MAY <= 4000

minOutJUN: output_JUN >= 1750

maxOutJUN: output_JUN <= 3500

_C2: output_JAN + storage_000 - storage_JAN = 1000

_C3: output_FEB - storage_FEB + storage_JAN = 4500

_C4: output_MAR + storage_FEB - storage_MAR = 6000

_C5: output_APR - storage_APR + storage_MAR = 5500

_C6: output_MAY + sto

In [49]:
# Optimize

prob.solve()

# Print the status of the solved LP
print("Status = %s" % LpStatus[prob.status])

Status = Optimal


In [51]:
# Print the value of the variables at the optimum
for i in months:
    print("%s | o= %f | s= %f" % (i, s[i].varValue, o[i].varValue))

# Print the value of the objective
print("Objective = %f" % value(prob.objective))

000 | o= 2750.000000 | s= 0.000000
JAN | o= 5750.000000 | s= 4000.000000
FEB | o= 4750.000000 | s= 3500.000000
MAR | o= 2750.000000 | s= 4000.000000
APR | o= 1500.000000 | s= 4250.000000
MAY | o= 2000.000000 | s= 4000.000000
JUN | o= 1500.000000 | s= 3500.000000
Objective = 6209403.125000
